# Advanced prompts

In this notebook, we will explore some advanced prompt patterns:  
+ Few-shots prompt
+ Chain of Thought prompt
+ Program-Aided Language Models (PAL)
+ REACT
+ CO-STAR

## Setup
#### Load the API key and relevant Python libraries.

In [1]:
%pip install -qU openai

/home/renato/Documents/Repos/raiffeisen-agentic-ai-training/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

import openai
from dotenv import load_dotenv
from openai import AzureOpenAI

In [3]:
# Load environment variables from .env file
load_dotenv()

llm_endpoint = "https://aa-dsa-training-msca.openai.azure.com/"
llm_model_name = "gpt-5-nano"
llm_deploy_name = "gpt-5-nano"

subscription_api_key = os.getenv("OPENAI_API_KEY")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_key=subscription_api_key,
    api_version=api_version,
    azure_endpoint=llm_endpoint,
)


In [4]:
def ask(prompt: str) -> str | None:
    """Send a chat prompt to the Azure OpenAI model and return the assistant's reply.

    Args:
        prompt (str): The user prompt to send to the model.

    Returns:
        str | None: The assistant response text on success, otherwise None.
    """
    try:
        chat_completion = client.chat.completions.create(
            model=llm_deploy_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ],
        )
        return chat_completion.choices[0].message.content
    except openai.AuthenticationError as e:
        print(f"OpenAI API returned an Authentication Error: {e}")
    except openai.APIConnectionError as e:
        print(f"Failed to connect to OpenAI API: {e}")
    except openai.BadRequestError as e:
        print(f"Invalid Request Error: {e}")
    except openai.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
    except openai.InternalServerError as e:
        print(f"Service Unavailable: {e}")
    except openai.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
    except Exception as e:
        print(f"An exception has occured: {e}")
    return None

### Few-shots prompt

Few-shots prompting means that along with the instruction, you input demonstrations of how the task can be solved. 

##### Zero-shot prompting (without few-shot prompting):

In [5]:
response = ask(
    "Please analyze the sentiment of the following sentence: 'This phone lasts all day on a single charge. It’s amazing!'"
)

print(response)

- Overall sentiment: Positive
- Polarity: Positive
- Intensity: High/strong
- Key cues driving sentiment:
  - "This phone lasts all day on a single charge" expresses satisfaction with battery life
  - "It’s amazing!" reinforces strong positive emotion
- Subjectivity: Opinionated (subjective evaluation)
- Suggested sentiment score (approximate): around +0.9 on a -1 to 1 scale


##### With few-shot prompting:

In [6]:
query = """
I will provide you with a series of sentences. 
After each sentence, please analyze the sentiment expressed and categorize it as “Positive,” “Negative,” or “Neutral.”

Q) "I absolutely love the friendly service at that restaurant." 
A) {Sentiment: "Positive"}
Q) "The movie was a complete waste of time and money." 
A) {Sentiment: "Negative"}
Q) "It’s a typical day, nothing much happening." 
A) {Sentiment: "Neutral"}

Now, please analyze the sentiment of the following sentence:

"This phone lasts all day on a single charge. It’s amazing!"
"""

response = ask(query)
print(response)

{Sentiment: "Positive"}


### Chain of Thought prompt

The main idea of Chain-of-Thought (CoT) prompting is that LLMs handle complex tasks better if they are divided into intermediate steps.

##### Without Chain of Thought:

In [7]:
query = """
Question: What will be the atomic number of the next element to be added to the periodic table of elements?
"""

response = ask(query)
print(response)

119.

Note: The next element to be added would have atomic number 119 (temporary name Ununennium, symbol Uue). As of now, it has not yet been synthesized.


##### With Chain of Thought:

In [8]:
query = """
- Every element we add to the periodic table of elements has a bigger atomic number comparing to the successor.
- The next element's atomic number adds one to the atomic number of previous one in the periodic table.
- Oganesson, the last element to be added, has the atomic number of 118.

Question: What will be the atomic number of the next element to be added to the periodic table of elements?
"""

response = ask(query)
print(response)

119

Reason: Each new element has an atomic number one greater than the previous. Oganesson is 118, so the next element would have atomic number 118 + 1 = 119. (The next element has not yet been discovered; it’s often referred to by the temporary name ununennium, Uue.)


### [Program-Aided Language Models (PAL)](https://reasonwithpal.com/)

PAL is a method, where language models generate programs from natural language descriptions, which are then executed by a programmatic runtime like a Python interpreter. That leads to a more accurate result.

##### Without PAL:

In [9]:
query = "There were nine computers in the server room. Five more computers were installed each day, since Monday. How many computers are now in the server room?"
response = ask(query)
print(response)

We can’t determine the exact number without knowing how many days have passed since Monday.

If d is the number of days from Monday to today inclusive (i.e., Monday counts as 1), then:
- New total = 9 + 5d

Examples (assuming today is within the same week):
- Monday: d = 1 → 9 + 5 = 14
- Tuesday: d = 2 → 9 + 10 = 19
- Thursday: d = 4 → 9 + 20 = 29
- Friday: d = 5 → 9 + 25 = 34

If you tell me what day it is today (or how many days have passed since Monday), I’ll give the exact total.


##### With PAL:

In [10]:
query = """
There were nine computers in the server room last Monday. 
Five more computers were installed each day, until today. 
How many computers are now in the server room?
create a Python code to answer the question, execute it, and print the result
Explain each step.
"""

response = ask(query)
print(response)

Here's a simple Python approach to solve the problem, along with an explanation of each step. Since the problem statement doesn’t specify how many days have passed since last Monday, I’ll show the calculation for the common assumption that today is Friday (4 days after Monday). You can change the days variable if today is a different day.

Explanation of steps:
- Step 1: Identify the starting number of computers. It’s 9.
- Step 2: Determine how many days have passed since last Monday. If today is Friday, that’s 4 days.
- Step 3: Each day, 5 more computers are installed. So total additional computers = days_since_monday * 5.
- Step 4: Total computers now = starting number + total additional computers.
- Step 5: Implement this in Python and print the result.

Python code:
days_since_monday = 4  # adjust if today is a different day after Monday
initial = 9
added_per_day = 5

total_computers = initial + days_since_monday * added_per_day
print("Total computers now:", total_computers)

Execu

### [REACT](https://arxiv.org/pdf/2210.03629)

ReAct is a prompting technique that combines reasoning, that  helps the model induce, track, and update action plans as well as handle
exceptions, and actions, that allow it to interface with and gather additional information
from external sources.

##### Without REACT:

In [9]:
query = """
Emma took a 80 minutes plane ride to Seattle. After spending a quarter of hour in a cab, She then took a two hours train ride to Portland, 
but the train was 5 minutes late. Initially, she would only wait for 15 minutes in the train station before the train departs. 
She then took an Uber for the airport, which took 1/2 of hour. Then, after 1 hour waiting in the airport, she took a plane to Vancouver 
that lasted half of the time she spent on the first transportation mode. 
How long did it take her to get to Vancouver?
"""

response = ask(query)
print(response)

6 hours 5 minutes.

Reasoning:
- First plane to Seattle: 80 minutes
- Cab: 15 minutes
- Train Seattle to Portland: 2 hours = 120 minutes, plus 5 minutes late means waiting time at station becomes 15 + 5 = 20 minutes
- Train wait: 20 minutes
- Uber to airport: 30 minutes
- Airport wait: 60 minutes
- Plane Seattle to Vancouver: half of the first leg's time = 40 minutes

Total time = 80 + 15 + 120 + 20 + 30 + 60 + 40 = 365 minutes = 6 hours 5 minutes.


##### With REACT:

In [11]:
query = """
Emma took a 80 minutes plane ride to Seattle. After spending a quarter of hour in a cab, She then took a two hours train ride to Portland, 
but the train was 5 minutes late. Initially, she would only wait for 15 minutes in the train station before the train departs. 
She then took an Uber for the airport, which took 1/2 of hour. Then, after 1 hour waiting in the airport, she took a plane to Vancouver 
that lasted half of the time she spent on the first transportation mode. 
How long did it take her to get to Vancouver? 
Use the REACT framework to show your reasoning.
"""

response = ask(query)
print(response)

R (Read the problem)
- We need total time from the start of Emma’s first trip (the 80-minute plane to Seattle) until she arrives in Vancouver.

E (Extract the data)
- First plane: 80 minutes
- Cab after arrival: 15 minutes
- Train to Portland: 2 hours = 120 minutes
- Train delay: 5 minutes late
- Waiting at train station originally planned: 15 minutes
  - Since the train is late by 5 minutes, actual wait becomes 15 + 5 = 20 minutes
- Uber to the airport: 0.5 hour = 30 minutes
- Waiting in the airport: 1 hour = 60 minutes
- Plane to Vancouver: duration = half of first transport time = 80 / 2 = 40 minutes

A (Analysis/Calculation)
- Total time = sum of all segments:
  - 80 (Seattle flight)
  - 15 (cab)
  - 20 (train wait, including delay)
  - 120 (train ride)
  - 30 (Uber)
  - 60 (airport wait)
  - 40 (Vancouver flight)
- Compute: 80 + 15 + 20 + 120 + 30 + 60 + 40 = 365 minutes
- Convert: 365 minutes = 6 hours and 5 minutes

C (Conclusion)
- It took 6 hours and 5 minutes for Emma to get 

### CO-STAR  

Effective prompt structuring is crucial for eliciting optimal responses from an LLM. The CO-STAR framework is a handy template for structuring prompts.  

It considers all the key aspects that influence the effectiveness and relevance of an LLM’s response, leading to more optimal responses.  

##### Without CO-STAR:

In [12]:
query = "Write a LinkedIn post to advertise Raiffeisen new product called voiceBot, to allow electronic banking transactions via voice."

response = ask(query)
print(response)

Raiffeisen is excited to unveil voiceBot — our new voice-powered solution for electronic banking transactions. Banking, now powered by your voice: simple, faster, and safer.

What you can do with voiceBot:
- Check balances and recent transactions
- Transfer funds between your accounts or to another beneficiary
- Pay bills and schedule payments
- Set up recurring transfers and payment reminders
- Get real-time insights and transaction history
- Find branches, ATMs, or contact support with a quick voice query

Security first: voiceBot operates within Raiffeisen’s trusted security framework, featuring voice biometrics, device binding, and multi-factor authentication to ensure every action is properly authorized. Your privacy is protected in line with our privacy standards and applicable regulations.

Ready to experience hands-free banking? If you’re a Raiffeisen customer, look out for the app update or join our early-access program. Learn more at [link], or drop a comment to request a dem

##### With CO-STAR:

In [13]:
query = """
# CONTEXT #
I want to advertise my company’s new product. My company’s name is Raiffeisen and the product is called VoiceBot, that allow electronic banking transactions via voice.

# OBJECTIVE #
Create a LinkedIn post for me, which aims to get people to click on the link to read it.

# STYLE #
Follow the writing style of successful companies on the same sector that advertise similar products.

# TONE #
Persuasive

# AUDIENCE #
My company’s audience profile on LinkedIn is typically the older generation. Tailor your post to target what this audience typically looks out for in hair products.

# RESPONSE #
The LinkedIn post
"""

response = ask(query)
print(response)

Raiffeisen introduces VoiceBot — banking that talks back. Manage your money with simple voice commands: check balances, review recent activity, send transfers, and schedule payments — all without touching a screen.

Why this matters to you:
- Hands-free control for everyday banking, with clear prompts and confirmations you can trust
- Security you can rely on: robust encryption, privacy-first design, and bank-grade protections
- Accessibility you deserve: straightforward language, easy-to-understand steps, and dependable support

Designed with comfort and confidence in mind for all generations, especially those who value reliability and ease of use in digital banking.

Read the full story to see how VoiceBot can fit into your daily routine: https://www.raiffeisen.com/voicebot

#VoiceBot #Raiffeisen #DigitalBanking #VoiceBanking #CustomerExperience
